In [28]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from scipy.io import loadmat
import numpy as np
import pandas as pd
from operator import itemgetter
import csv
from matFileFunctions import *
from trainingModels import *

In [5]:
DATA = []
CLASS = []

matFileNames = ["data-starplus-04799-v7.mat", "data-starplus-04820-v7.mat", "data-starplus-04847-v7.mat", "data-starplus-05675-v7.mat", "data-starplus-05680-v7.mat", "data-starplus-05710-v7.mat"]
DATA, DATA1 = [], []
CLASS, CLASS1 = [], []

for matFileName in matFileNames:
    matFile = loadmat(matFileName)
    regionOfInterest = getRegionOfInterest(matFile)
    importantColumns = []
    meta = getMeta(matFile)
    for r in regionOfInterest:
        importantColumns.extend(meta['rois'][r]['columns'])
    importantColumns = list(set(map(lambda x: x-1, importantColumns)))
    print(len(importantColumns))
    ## importantColumns has all required columns from data ##

    d,c = matFileToData(matFile)  # data[:17] data[17:]
    #print(len(d[0]))
    DATA.append(d)
    CLASS.append(c)     # P:-1 | S:1
    
    d,c = matFileToDataWithImportantColumns(matFile, importantColumns)  # data[:9] data[17:26]
    DATA1.append(d)
    CLASS1.append(c)    # P:-1 | S:1 
    
    

1874
1888
1714
2239
2230
1883


In [21]:
##  Whole set [:17], [18:]
for i in range(6):
    print("------------------------For patient ",i,"---------------------------")
    data = np.array(DATA[i])
    label = np.array(CLASS[i])
    k=2
    kf = KFold(k, shuffle=True)
    kf.get_n_splits(data)
    df = pd.DataFrame()
    D = []
    c=0
    for train_index, test_index in kf.split(data):
        print("k=",c+1)
        c+=1
        trainData = data[train_index]
        trainLable = label[train_index]
        testData = data[test_index]
        testLable = label[test_index]
        D.append(trainAllModels(trainData, trainLable, testData, testLable))
    [avg_precision, avg_recall, avg_f1, avg_accuracy] = calculateAverageScores(D)
    print("%-16s | %8s | %8s | %8s | %8s"%('','Precision','Recall','F1','Accuracy'))
    for key,val in avg_precision.items():
          print("%16s | %5f  | %5f | %5f | %5f"%(key, avg_precision[key], avg_recall[key], avg_f1[key], avg_accuracy[key]))
    print("\n------------------------------------------------------------------------------------------------------")

------------------------For patient  0 ---------------------------
k= 1
k= 2
                 | Precision |   Recall |       F1 | Accuracy
   NeuralNetwork | 1.000000  | 0.422131 | 0.593368 | 0.422131
              RF | 1.000000  | 0.951275 | 0.974895 | 0.951275
             SVM | 1.000000  | 0.852459 | 0.920322 | 0.852459
             KNN | 1.000000  | 0.826047 | 0.904737 | 0.826047
              NB | 1.000000  | 0.886612 | 0.939243 | 0.886612
              GB | 1.000000  | 0.627049 | 0.770759 | 0.627049
             XGB | 1.000000  | 0.963570 | 0.981433 | 0.963570
              LR | 1.000000  | 0.981330 | 0.990573 | 0.981330

------------------------------------------------------------------------------------------------------
------------------------For patient  1 ---------------------------
k= 1
k= 2
                 | Precision |   Recall |       F1 | Accuracy
   NeuralNetwork | 1.000000  | 0.415756 | 0.586916 | 0.415756
              RF | 1.000000  | 0.880237 | 0.936303 | 0.88023

In [22]:
## Partial [:8], [17:24]
D, C = DATA1, CLASS1
for i in range(6):
    print("------------------------For patient ",i,"---------------------------")
    data = np.array(DATA1[i])
    label = np.array(CLASS1[i])
    k=2
    kf = KFold(k, shuffle=True)
    kf.get_n_splits(data)
    df = pd.DataFrame()
    D = []
    c=0
    for train_index, test_index in kf.split(data):
        print("k=",c+1)
        c+=1
        trainData = data[train_index]
        trainLable = label[train_index]
        testData = data[test_index]
        testLable = label[test_index]
        D.append(trainAllModels(trainData, trainLable, testData, testLable))
    [avg_precision, avg_recall, avg_f1, avg_accuracy] = calculateAverageScores(D)
    print("%-16s | %8s | %8s | %8s | %8s"%('','Precision','Recall','F1','Accuracy'))
    for key,val in avg_precision.items():
          print("%16s | %5f  | %5f | %5f | %5f"%(key, avg_precision[key], avg_recall[key], avg_f1[key], avg_accuracy[key]))
    print("\n------------------------------------------------------------------------------------------------------")

------------------------For patient  0 ---------------------------
k= 1
k= 2
                 | Precision |   Recall |       F1 | Accuracy
   NeuralNetwork | 1.000000  | 0.141667 | 0.239410 | 0.141667
              RF | 1.000000  | 0.812500 | 0.896318 | 0.812500
             SVM | 1.000000  | 0.698611 | 0.822340 | 0.698611
             KNN | 1.000000  | 0.713889 | 0.833060 | 0.713889
              NB | 1.000000  | 0.826389 | 0.904927 | 0.826389
              GB | 1.000000  | 0.580556 | 0.734618 | 0.580556
             XGB | 1.000000  | 0.815278 | 0.898234 | 0.815278
              LR | 1.000000  | 0.887500 | 0.940392 | 0.887500

------------------------------------------------------------------------------------------------------
------------------------For patient  1 ---------------------------
k= 1
k= 2
                 | Precision |   Recall |       F1 | Accuracy
   NeuralNetwork | 1.000000  | 0.220833 | 0.358203 | 0.220833
              RF | 1.000000  | 0.852778 | 0.920343 | 0.85277

In [36]:

for i in range(6):
    print("------------------------For patient ",i,"---------------------------")
    data = np.array(DATA2[i])
    label = np.array(CLASS2[i])
    k=2
    kf = KFold(k, shuffle=True)
    kf.get_n_splits(data)
    df = pd.DataFrame()
    D = []
    c=0
    for train_index, test_index in kf.split(data):
        print("k=",c+1)
        c+=1
        trainData = data[train_index]
        trainLable = label[train_index]
        testData = data[test_index]
        testLable = label[test_index]
        D.append(trainAllModels(trainData, trainLable, testData, testLable))
    [avg_precision, avg_recall, avg_f1, avg_accuracy] = calculateAverageScores(D)
    print("%-16s | %8s | %8s | %8s | %8s"%('','Precision','Recall','F1','Accuracy'))
    for key,val in avg_precision.items():
          print("%16s | %5f  | %5f | %5f | %5f"%(key, avg_precision[key], avg_recall[key], avg_f1[key], avg_accuracy[key]))
    print("\n------------------------------------------------------------------------------------------------------")

,0,1,2,3,4,5,6,7,8,9,...,4939,4940,4941,4942,4943,4944,4945,4946,4947,4948
count,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,...,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000
mean,-0.560689,0.751394,0.119059,-0.448414,0.076310,0.179938,-0.071851,0.718527,0.675406,0.356115,...,-0.330049,0.128734,1.030396,4.821306,8.692990,0.084212,0.196764,1.458521,5.281262,6.683805
std,4.182890,2.599053,2.224981,2.599277,2.523181,2.714312,3.679427,2.635009,2.230483,2.143168,...,2.263616,2.291189,2.574532,5.993457,9.784131,2.579239,2.731679,3.388441,6.590254,8.451648
min,-17.001065,-7.766041,-6.696873,-8.620011,-8.838385,-9.001067,-11.284131,-7.865476,-6.138002,-6.543656,...,-7.196976,-6.751903,-8.055461,-21.561208,-32.457287,-9.707933,-10.187750,-10.560726,-23.981004,-27.224991
25%,-3.439427,-1.072915,-1.324734,-2.259282,-1.726400,-1.671578,-2.646237,-1.072508,-0.862727,-1.081810,...,-1.834446,-1.429156,-0.695676,1.621243,3.402043,-1.644144,-1.546713,-0.749082,1.705664,2.332337
50%,-0.502346,0.680711,0.107023,-0.517970,0.107168,0.103155,-0.176562,0.614354,0.718839,0.359715,...,-0.337079,0.188741,1.064697,5.160678,9.443163,0.072489,0.185930,1.589427,5.712035,7.350570
75%,2.207963,2.473634,1.550980,1.327478,1.784194,1.981978,2.411597,2.516145,2.243395,1.760080,...,1.231162,1.688518,2.762210,8.670363,14.966005,1.834283,2.060633,3.755160,9.621676,12.169957
max,13.164478,10.299189,7.800011,8.627454,8.457861,9.033479,12.230697,8.591836,7.194956,6.905747,...,8.556630,7.772505,8.990197,22.805551,39.569348,7.644855,9.532714,12.488132,22.995238,31.160431


In [43]:
DATA = []
CLASS = []

matFileNames = ["data-starplus-04799-v7.mat", "data-starplus-04820-v7.mat", "data-starplus-04847-v7.mat", "data-starplus-05675-v7.mat", "data-starplus-05680-v7.mat", "data-starplus-05710-v7.mat"]

for matFileName in matFileNames:
    Data,Class = [],[]
    matFile = loadmat(matFileName)
    regionOfInterest = getRegionOfInterest(matFile)
    importantColumns = []
    meta = getMeta(matFile)
    for r in regionOfInterest:
        importantColumns.extend(meta['rois'][r]['columns'])
    importantColumns = list(set(map(lambda x: x-1, importantColumns)))
    print(len(importantColumns))
    ## importantColumns has all required columns from data ##
    for roi in importantColumns:
        d,c = singleImportantColumn(matFile, [roi])  # data[:17] data[17:]
        print("Patient ",i)
        l=int(len(d)*0.8)
        trainData, testData = d[:l], d[l:]
        trainLable, testLable = c[:l], c[l:]

        model = svm.SVC(kernel='poly', C=0.5)
        model.fit(trainData, trainLable)
        predict = model.predict(testData)
        lrScore = calculateScores(testLable, predict)
        print("precision: {}, recall: {}, F1: {}, accuracy: {}".format(lrScore[0],lrScore[1],lrScore[2],lrScore[3]))
        break
    break
    

    

4949
Patient  1
precision: -1, recall: -1, F1: -1, accuracy: -1


In [44]:
for i,j in zip(testLable, predict):
    print(i, j)
#for i in DATA[0][0]:
#    print(i)

1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1


In [32]:
for i in range(6):
    print("Patient ",i)
    l=int(len(DATA[0][i])*0.8)
    trainData, testData = DATA[0][i][:l], DATA[0][i][l:]
    trainLable, testLable = CLASS[0][i][:l], CLASS[0][i][l:]
    
    lrModel = LogisticRegression(random_state=0, solver='lbfgs').fit(trainData, trainLable)
    predict = lrModel.predict(testData)
    lrScore = calculateScores(testLable, predict)
    print("precision: {}, recall: {}, F1: {}, accuracy: {}".format(lrScore[0],lrScore[1],lrScore[2],lrScore[3]))
    
    

Patient  0
precision: -1, recall: -1, F1: -1, accuracy: -1
Patient  1
precision: -1, recall: -1, F1: -1, accuracy: -1
Patient  2
precision: -1, recall: -1, F1: -1, accuracy: -1
Patient  3
precision: -1, recall: -1, F1: -1, accuracy: -1
Patient  4
precision: -1, recall: -1, F1: -1, accuracy: -1
Patient  5
precision: -1, recall: -1, F1: -1, accuracy: -1


In [33]:
for i,j in zip(testLable, predict):
    print(i, j)

1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
